# Generate Dataset

## Install library yang diperlukan

In [ ]:
!pip install simple_image_download

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.


## Buat folder train dan val

In [ ]:
import os
train_dir = os.path.join("dataset", "train")
val_dir = os.path.join("dataset", "validation")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

## Download images dari google

In [ ]:
from simple_image_download import simple_image_download as simp
response = simp.simple_image_download
lst=['mobil', 'motor', 'sepeda', 'jam tangan', 'kacamata', 'topi',
     'payung', 'handphone', 'dompet', 'kunci',
     'uang', 'kantong plastik', 'tas', 'buku',
     'ransel', 'pensil', 'pulpen', 'kertas', 'kalkulator',
     'sepatu', 'kipas angin', 'kompor', 'mesin cuci',
     'sendok', 'mangkok', 'garpu', 'gelas', 'pisau', 'kursi', 'meja',
     'kasur', 'televisi']
for rep in lst:
 response().download(rep, 200)

KeyboardInterrupt: 

## Pindahkan file gambar dari folder simple_images ke folder dataset train dan validation

In [ ]:
import shutil
import random

In [ ]:
# Replace with your actual folder paths
source_dir = "/content/simple_images"  # Directory containing subfolders X, Y, and Z
dataset_dir = "/content/dataset"  # Directory containing train and validation subfolders

In [ ]:
split_ratio = 0.8

In [ ]:
def move_files(source_path, destination_path, split_ratio):
    # Get a list of all files in the source folder
    files = os.listdir(source_path)

    # Shuffle the list of files to randomize the order
    random.shuffle(files)

    # Calculate the number of files to move to training
    num_train_files = int(len(files) * split_ratio)

    # Move files to train and validation based on the split ratio
    for i, filename in enumerate(files):
        if i < num_train_files:
            shutil.move(os.path.join(source_path, filename), train_subfolder_path)
        else:
            shutil.move(os.path.join(source_path, filename), validation_subfolder_path)

In [ ]:
for subfolder_name in lst:
    # Construct the full paths for source and destination subfolders
    source_subfolder_path = os.path.join(source_dir, subfolder_name)
    train_subfolder_path = os.path.join(dataset_dir, "train", subfolder_name)
    validation_subfolder_path = os.path.join(dataset_dir, "validation", subfolder_name)

    # Create the train and validation subfolders if they don't exist
    os.makedirs(train_subfolder_path, exist_ok=True)
    os.makedirs(validation_subfolder_path, exist_ok=True)

    # Move the specified percentage of files to train and validation
    move_files(source_subfolder_path, train_subfolder_path, split_ratio)
    move_files(source_subfolder_path, validation_subfolder_path, 1 - split_ratio)

# Download dataset raw (zip)

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("dataset", 'zip', "dataset")
files.download("dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Clean dataset manual untuk cek file corrupt, lalu upload ke drive

# Load cleaned dataset

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/Capstone Project/dataset clean/train'
val_dir = '/content/drive/MyDrive/Capstone Project/dataset clean/validation'

# Model building

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

Found 10862 images belonging to 71 classes.


In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

Found 2736 images belonging to 71 classes.


In [ ]:
base_model = ResNet50V2(weights='imagenet', include_top=False)

94668760/94668760 [==============================] - 5s 0us/step


In [ ]:
for layer in base_model.layers[:-1]:
  layer.trainable = False

In [ ]:
#Add new layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(71, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=x)

#Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#Callback
checkpoint = ModelCheckpoint('model72class.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

#Training
model.fit(
    train_generator,
    steps_per_epoch=50,
    epochs=50,  # Coba jumlah epoch yang lebih tinggi dengan early stopping
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint, early_stopping])

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


50/50 [==============================] - ETA: 0s - loss: 3.7670 - accuracy: 0.2212 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - 2701s 54s/step - loss: 3.7670 - accuracy: 0.2212 - val_loss: 2.2397 - val_accuracy: 0.5329
Epoch 2/50
50/50 [==============================] - 653s 13s/step - loss: 2.0897 - accuracy: 0.5063 - val_loss: 1.1279 - val_accuracy: 0.7460
Epoch 3/50
50/50 [==============================] - 660s 13s/step - loss: 1.5316 - accuracy: 0.6212 - val_loss: 0.8084 - val_accuracy: 0.8001
Epoch 4/50
50/50 [==============================] - 510s 10s/step - loss: 1.2037 - accuracy: 0.6825 - val_loss: 0.6478 - val_accuracy: 0.8344
Epoch 5/50
50/50 [==============================] - 519s 10s/step - loss: 1.0982 - accuracy: 0.7188 - val_loss: 0.5841 - val_accuracy: 0.8523
Epoch 6/50
50/50 [==============================] - 464s 9s/step - loss: 0.8857 - accuracy: 0.7750 - val_loss: 0.5317 - val_accuracy: 0.8516
Epoch 7/50
50/50 [==============================] - 416s 8s/step - loss: 0.7702 - accuracy: 0.7788 - val_loss: 0.5151 - val_accuracy: 0.8480
Epoch 8/50
50/50 [

### Print index dataset (label)

In [ ]:
class_indices = train_generator.class_indices
print(class_indices)

{'bantal': 0, 'beruang': 1, 'botol minum': 2, 'buah belimbing': 3, 'buah kelengkeng': 4, 'buah kesemek': 5, 'buah leci': 6, 'buah manggis': 7, 'buah markisa': 8, 'buah naga': 9, 'buah nangka': 10, 'buah pepaya': 11, 'buah semangka': 12, 'buah sirsak': 13, 'burung': 14, 'dompet': 15, 'durian': 16, 'gajah': 17, 'garpu': 18, 'gelas': 19, 'gunting': 20, 'jam dinding': 21, 'jam tangan': 22, 'jeruk bali': 23, 'kacamata': 24, 'kangguru': 25, 'kantong plastik': 26, 'kasur': 27, 'katak': 28, 'kertas': 29, 'komputer': 30, 'korek api': 31, 'kucing': 32, 'kulkas': 33, 'kumbang': 34, 'kunci (1)': 35, 'kupu-kupu': 36, 'kursi': 37, 'langit': 38, 'laptop': 39, 'lebah': 40, 'lemari': 41, 'mangkuk': 42, 'meja': 43, 'mesin jahit': 44, 'mobil': 45, 'motor': 46, 'paus': 47, 'payung': 48, 'penanda jalan': 49, 'penggaris': 50, 'pensil (1)': 51, 'piring': 52, 'pisau': 53, 'pot bunga': 54, 'rambutan': 55, 'ransel': 56, 'salak': 57, 'sapi': 58, 'sapu': 59, 'sendok': 60, 'sepeda': 61, 'setrika': 62, 'singa': 63,

In [ ]:
import csv
# Tentukan nama file CSV yang ingin Anda buat
csv_filename = 'class_indices.csv'

# Buka file CSV untuk menulis
with open(csv_filename, 'w', newline='') as csvfile:
    # Tentukan header untuk file CSV
    fieldnames = ['class_name', 'class_index']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Tulis header
    writer.writeheader()

    # Tulis setiap baris dengan nama kelas dan indeksnya
    for class_name, class_index in class_indices.items():
        writer.writerow({'class_name': class_name, 'class_index': class_index})

# Mengconvert TF keras ke TFLite

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input

In [ ]:
#Load model .h5
keras_model_path = '/content/drive/MyDrive/Capstone Project/model/model72class.h5'
model = load_model(keras_model_path)

In [ ]:
#Ubah input layer
input_layer = Input(shape=(224, 224, 3))
new_model = Model(inputs=input_layer, outputs=model(input_layer))

In [ ]:
#Cek input shape yang baru
print("New model input shape:", new_model.input_shape)

New model input shape: (None, 224, 224, 3)


In [ ]:
#Konversi model ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

In [ ]:
#Simpan model TFLite
tflite_model_path = '/content/drive/MyDrive/Capstone Project/model/model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)